In [98]:
from datetime import date
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine

# Database connections 

In [99]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [100]:
# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

# Crear los SQLAlchemy Engine
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

# SQL Query

In [101]:
query = """
SELECT 
    s.id AS servicio_id,
    os.cliente_id,
    sede.sede_id,
    COALESCE(s.mensajero3_id, COALESCE(s.mensajero2_id, s.mensajero_id)) AS mensajero_final_id,
    es.fecha AS fecha_estado,
    es.hora AS hora_estado
FROM mensajeria_servicio s
JOIN mensajeria_estadosservicio es 
    ON s.id = es.servicio_id
JOIN mensajeria_origenservicio os 
    ON s.origen_id = os.id
JOIN sede 
    ON sede.cliente_id = os.cliente_id 
    AND sede.ciudad_id = os.ciudad_id
ORDER BY s.id, es.fecha, es.hora


"""

# Extract

In [102]:
# Leer datos y dimensiones
df = pd.read_sql(query, fuente_conn)
dim_fecha = pd.read_sql_table('dim_fecha', bodega_conn)
dim_cliente = pd.read_sql_table('dim_cliente', bodega_conn)
dim_mensajero = pd.read_sql_table('dim_mensajero', bodega_conn)
dim_sede = pd.read_sql_table('dim_sede', bodega_conn)

# Transformations

In [103]:
# Convertir fecha_estado a datetime
df['fecha_estado'] = pd.to_datetime(df['fecha_estado']).dt.date
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha']).dt.date

In [104]:
# Calcular tiempo de entrega por servicio
def calcular_tiempo_entrega(grupo):
    estados_ordenados = grupo.sort_values(['fecha_estado', 'hora_estado'])
    if len(estados_ordenados) >= 2:
        inicio = pd.to_datetime(f"{estados_ordenados.iloc[0]['fecha_estado']} {estados_ordenados.iloc[0]['hora_estado']}")
        fin = pd.to_datetime(f"{estados_ordenados.iloc[-1]['fecha_estado']} {estados_ordenados.iloc[-1]['hora_estado']}")
        return (fin - inicio).total_seconds() / 3600  # Convertir a horas
    return None

df_tiempo_entrega = df.groupby('servicio_id').apply(calcular_tiempo_entrega).reset_index(name='tiempo_entrega')


C:\Users\HP\AppData\Local\Temp\ipykernel_29608\1054194721.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_tiempo_entrega = df.groupby('servicio_id').apply(calcular_tiempo_entrega).reset_index(name='tiempo_entrega')


In [105]:
# Obtener solo el primer estado de cada servicio para la fecha
df_servicios = df.sort_values(['fecha_estado', 'hora_estado']).groupby('servicio_id').first().reset_index()


In [106]:
# Combinar con los tiempos de entrega
df_servicios = df_servicios.merge(df_tiempo_entrega, on='servicio_id', how='left')


In [107]:
# Realizar los merges con las dimensiones
hecho_diario = df_servicios.merge(
    dim_fecha[['key_dim_fecha', 'fecha']], 
    left_on='fecha_estado', 
    right_on='fecha',
    how='left'
)

hecho_diario = hecho_diario.merge(
    dim_cliente[['key_dim_cliente', 'cliente_id']], 
    on='cliente_id',
    how='left'
)

hecho_diario = hecho_diario.merge(
    dim_mensajero[['key_dim_mensajero', 'mensajero_id']], 
    left_on='mensajero_final_id',
    right_on='mensajero_id',
    how='left'
)

hecho_diario = hecho_diario.merge(
    dim_sede[['key_dim_sede', 'id_sede']],  
    left_on='sede_id',  
    right_on='id_sede',  
    how='left'
)



In [108]:
# Seleccionar columnas finales
columnas_finales = [
    'servicio_id',
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_mensajero',
    'key_dim_sede',
    'tiempo_entrega'
]

hecho_diario = hecho_diario[columnas_finales]

In [109]:
# Agregar fecha de carga
hecho_diario['saved'] = date.today()


# Verifications

In [110]:
# Verificaciones
print("\nInformación del DataFrame:")
print(hecho_diario.info())

print("\nEstadísticas de tiempo de entrega:")
print(hecho_diario['tiempo_entrega'].describe())

print("\nVerificar valores nulos:")
print(hecho_diario.isnull().sum())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28401 entries, 0 to 28400
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   servicio_id        28401 non-null  int64  
 1   key_dim_fecha      28401 non-null  int64  
 2   key_dim_cliente    28401 non-null  int64  
 3   key_dim_mensajero  27677 non-null  float64
 4   key_dim_sede       28401 non-null  int64  
 5   tiempo_entrega     28320 non-null  float64
 6   saved              28401 non-null  object 
dtypes: float64(2), int64(4), object(1)
memory usage: 1.5+ MB
None

Estadísticas de tiempo de entrega:
count    28320.000000
mean         7.369806
std         80.470387
min          0.000000
25%          0.953889
50%          1.677361
75%          2.891875
max       4871.641667
Name: tiempo_entrega, dtype: float64

Verificar valores nulos:
servicio_id            0
key_dim_fecha          0
key_dim_cliente        0
key_dim_mensaje

# Load

In [111]:
# Guardar en la bodega
hecho_diario.to_sql(
    'hecho_entrega_servicio_diaria', 
    bodega_conn, 
    if_exists='replace', 
    index='key_hecho_entrega_servicio_diaria'
)

401

# SQL Querys to response requirements

In [112]:
#Pregunta 1: ¿En qué meses del año los clientes solicitan más servicios de mensajería?

'''
SELECT 
    df.mes,
    COUNT(h.servicio_id) AS total_servicios
FROM hecho_entrega_servicio_dia h
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY df.mes
ORDER BY total_servicios DESC;
'''

'\nSELECT \n    df.mes,\n    COUNT(h.servicio_id) AS total_servicios\nFROM hecho_entrega_servicio_dia h\nJOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha\nGROUP BY df.mes\nORDER BY total_servicios DESC;\n'

In [113]:
#Pregunta 2: ¿Cuáles son los días donde más solicitudes hay?

'''
SELECT 
    df.dia_semana,
    COUNT(h.servicio_id) AS total_servicios
FROM hecho_entrega_servicio_diaria h
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY df.dia_semana
ORDER BY total_servicios DESC;
'''

'\nSELECT \n    df.dia_semana,\n    COUNT(h.servicio_id) AS total_servicios\nFROM hecho_entrega_servicio_diaria h\nJOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha\nGROUP BY df.dia_semana\nORDER BY total_servicios DESC;\n'

In [114]:
#Pregunta 4: Número de servicios solicitados por cliente y por mes

'''
SELECT 
    dc.nombre AS nombre_cliente,
    df.mes,
    COUNT(h.servicio_id) AS total_servicios
FROM hecho_entrega_servicio_diaria h
JOIN dim_cliente dc ON h.key_dim_cliente = dc.key_dim_cliente
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY dc.nombre, df.mes
ORDER BY dc.nombre, df.mes;
'''

'\nSELECT \n    dc.nombre AS nombre_cliente,\n    df.mes,\n    COUNT(h.servicio_id) AS total_servicios\nFROM hecho_entrega_servicio_diaria h\nJOIN dim_cliente dc ON h.key_dim_cliente = dc.key_dim_cliente\nJOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha\nGROUP BY dc.nombre, df.mes\nORDER BY dc.nombre, df.mes;\n'

In [115]:
#Pregunta 6: ¿Cuáles son las sedes que más servicios solicitan por cada cliente?
'''
SELECT 
    dc.nombre AS nombre_cliente,
    ds.nombre_sede,
    COUNT(h.servicio_id) AS total_servicios
FROM hecho_entrega_servicio_diaria h
JOIN dim_cliente dc ON h.key_dim_cliente = dc.key_dim_cliente
JOIN dim_sede ds ON h.key_dim_sede = ds.key_dim_sede
GROUP BY dc.nombre, ds.nombre_sede
ORDER BY total_servicios DESC;

'''



'\nSELECT \n    dc.nombre AS nombre_cliente,\n    ds.nombre_sede,\n    COUNT(h.servicio_id) AS total_servicios\nFROM hecho_entrega_servicio_diaria h\nJOIN dim_cliente dc ON h.key_dim_cliente = dc.key_dim_cliente\nJOIN dim_sede ds ON h.key_dim_sede = ds.key_dim_sede\nGROUP BY dc.nombre, ds.nombre_sede\nORDER BY total_servicios DESC;\n\n'